# Execute

> TODO fill in description

In [ ]:
#| default_exp runtime

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio
from collections import deque
from types import MappingProxyType
from typing import Type, Optional, Union, Any, Tuple, Dict
from abc import ABC, abstractmethod

import fbdev
from fbdev.packet import Packet
from fbdev.port import PortType, PortSpec, ConfigPortSpec, PortTypeSpec, PortSpecCollection, BasePort, InputPort, ConfigPort, OutputPort, PortCollection
from fbdev.component import BaseComponent, ComponentFactory, PortSpec, BasePort, InputPort, OutputPort, PortCollection, PortType
from fbdev.graph import EdgeSpec, NodeSpec, Graph, ReadonlyGraph
from fbdev.exceptions import ComponentError, NodeError, LostPacketError
from fbdev.node import Node, GraphComponentFactory

In [ ]:
#|hide
from fbdev.component import func_component

In [ ]:
#|hide
show_doc(fbdev.runtime.NetRuntime)

---

### NetRuntime

>      NetRuntime ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class NetRuntime(ABC):
    @classmethod
    def from_graph(cls, graph:Graph):
        component_type = GraphComponentFactory.get_component(graph)
        net_spec = NodeSpec(component_type)
        net = Node(net_spec)
        return cls(net)
    
    @classmethod
    def execute_graph(cls, graph:Graph, *args, config={}, **kwargs):
        with cls.from_graph(graph) as netrun:
            return netrun.execute(*args, config, **kwargs)
    
    @classmethod
    async def async_execute_graph(cls, graph:Graph, *args, config={}, **kwargs):
        async with cls.from_graph(graph) as netrun:
            return await netrun.async_execute(*args, config=config, **kwargs)
    
    @abstractmethod
    def execute(self, *args, config={}, **kwargs):
        raise NotImplementedError()
    
    @abstractmethod
    async def async_execute(self, *args, config={}, **kwargs):
        raise NotImplementedError()
    
    @abstractmethod
    async def destroy(self):
        raise NotImplementedError()
    
    async def __aenter__(self):
        return self

    async def __aexit__(self, exc_type, exc_value, traceback):
        await self.destroy()
        
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        asyncio.run(self.destroy())

In [ ]:
#|hide
show_doc(fbdev.runtime.BatchExecutor)

---

### BatchExecutor

>      BatchExecutor (net:fbdev.node.Node)

*Executes a net like a batch process (input fed in the beginning, and no input during the execution, and output is returned at the end).*

In [ ]:
#|export
class BatchExecutor(NetRuntime):
    """Executes a net like a batch process (input fed in the beginning, and no input during the execution, and output is returned at the end)."""
    def __init__(self, net:Node):
        super().__init__()
        self._net:Node = net
    
    def _setup_execution_coro(self, *args, config={}, **kwargs):
        if not self._net.states.initialised.get(): self._net.initialise()
        net = self._net
        
        if net.states.running.get(): raise RuntimeError("Net is already running.")
        if not net.states.initialised.get(): raise RuntimeError("Net is not initialised.")
        
        # Fill in input args
        input = {**kwargs}
        for port_name, val in zip(net.port_specs.input.keys(), args):
            if port_name in input:
                raise ValueError(f"Multiple values provided for '{port_name}'.")
            input[port_name] = val
        missing_input_args = set(net.port_specs.input.keys()) - set(input.keys())
        if len(missing_input_args) > 0:
            raise ValueError(f"Missing values for ports {missing_input_args}.")
        
        # Check for unexpected input args
        extra_input_args = set(input.keys()) - set(net.port_specs.input.keys())
        if len(extra_input_args) > 0:
            raise ValueError(f"Unexpected values for inputs {extra_input_args}.")
        
        # Check for unexpected config args
        extra_config_args = set(config.keys()) - set(net.port_specs.config.keys())
        if len(extra_config_args) > 0:
            raise ValueError(f"Unexpected values for configs {extra_config_args}.")
                
        async def input_sender(port_type, port_name, val):
            await net.send_input(port_type, port_name, val)
            
        output = {}
        async def output_receiver(port_type, port_name):
            data = await net.receive_output(port_type, port_name)
            output[port_name] = data
        
        input_senders = [input_sender(PortType.INPUT, port_name, val) for port_name, val in input.items()]
        config_senders = [input_sender(PortType.CONFIG, port_name, val) for port_name, val in config.items()]
        output_receivers = [output_receiver(PortType.OUTPUT, port_name) for port_name in net.port_specs.output.keys()]
        
        async def main():
            net.run()
            await net._execute_with_exception_monitoring(*input_senders, *config_senders, *output_receivers)
            await net._execute_with_exception_monitoring(net.async_stop())
            
        return main(), output

    def execute(self, *args, config={}, **kwargs):
        """Note: this method cannot be run from within an event loop."""
        exec_coro, output = self._setup_execution_coro(*args, config=config, **kwargs)
        asyncio.run(exec_coro)
        return output
    
    async def async_execute(self, *args, config={}, **kwargs):
        exec_coro, output = self._setup_execution_coro(*args, config=config, **kwargs)
        await exec_coro
        return output
    
    async def destroy(self):
        if self._net is not None:
            await self._net.destroy()
            self._net = None
    
    def __del__(self):
        if self._net is not None:
            #TODO log a warning or handle the case where close() was not called
            pass

In [ ]:
@func_component()
def add_one(a:int) -> int:
    print("In add_one")
    return a+1

@func_component()
def copier(b:int) -> Tuple[int, int]:
    print("In copier")
    return b, b

@func_component()
def printer(c:int) -> None:
    print("In printer1:", c)
    
@func_component()
def sender(d:int):
    print("In sender")
    return d
    
g = Graph(PortSpecCollection(
    input=PortTypeSpec(in1=PortSpec(dtype=int)),
    output=PortTypeSpec(out=PortSpec(dtype=int))
))

add_one_id = g.add_node(NodeSpec(add_one))
copier_id = g.add_node(NodeSpec(copier))
printer_id = g.add_node(NodeSpec(printer))
sender_id = g.add_node(NodeSpec(sender))

g.connect_node_to_graph_port(add_one_id, PortType.INPUT, 'a',
                                        PortType.INPUT, 'in1')
g.connect_nodes(add_one_id, PortType.OUTPUT, 'out',
                copier_id, PortType.INPUT, 'b')
g.connect_nodes(copier_id, PortType.OUTPUT, 'out0',
                printer_id, PortType.INPUT, 'c')
g.connect_nodes(copier_id, PortType.OUTPUT, 'out1',
                sender_id, PortType.INPUT, 'd')
g.connect_node_to_graph_port(sender_id, PortType.OUTPUT, 'out',
                                        PortType.OUTPUT, 'out')

output = await BatchExecutor.async_execute_graph(g, 1)
print("Output:", output['out'])

out False None
In add_one
In copier
In printer1: 2
In sender
out >> <fbdev.packet.Packet object> True
Output: 2
